In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# GoogLeNet

In [4]:
def inception_module(x, filters):
  # Filters is a tuple that contains the number of filters for each branch
    f1, f2, f3, f4 = filters
  
  # Branch 1
    branch1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)
    
  # Branch 2
    branch2 = Conv2D(f2, (1, 1), padding='same', activation='relu')(x)
    branch2 = Conv2D(f2, (3, 3), padding='same', activation='relu')(branch2)
  
  # Branch 3
    branch3 = Conv2D(f3, (1, 1), padding='same', activation='relu')(x)
    branch3 = Conv2D(f3, (5, 5), padding='same', activation='relu')(branch3)
  
  # Branch 4
    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(f4, (1, 1), padding='same', activation='relu')(branch4)
  
  # Concatenate the outputs of the branches
    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
  
    return output
  
def GoogLeNet(input_shape, num_classes):
    input = tf.keras.layers.Input(shape=input_shape)

  # First Convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  
  # Second Convolutional layer
    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  
  # Inception 3a
    x = inception_module(x, filters=(64, 96, 128, 16))
  # Inception 3b
    x = inception_module(x, filters=(128, 128, 192, 32))
   
  # Max pooling with stride 2
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  
  # Inception 4a
    x = inception_module(x, filters=(192, 96, 208, 16))
  # Inception 4b
    x = inception_module(x, filters=(160, 112, 224, 24))
  # Inception 4c
    x = inception_module(x, filters=(128, 128, 256, 24))
  # Inception 4d
    x = inception_module(x, filters=(112, 144, 288, 32))
  # Inception 4e
    x = inception_module(x, filters=(256, 160, 320, 32))
  
  # Max pooling with stride 
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
  # Inception 5a
    x = inception_module(x, filters=(256, 160, 320, 32))
  # Inception 5b
    x = inception_module(x, filters=(384, 192, 384, 48))
    
  # global average pooling layer
    x = AveragePooling2D((7, 7), padding='same')(x)
    #x = Flatten()(x)
    
  # Fully Connected layer with linear. and. Dropout
    x = Dropout(0.4)(x)
    x = Dense(1024, activation='relu')(x)
    
    
  # softmax
    output_layer = Dense(num_classes, activation='softmax')(x)
    
  # define the model with input and output layers  
    model = tf.keras.Model(inputs=input, outputs=output_layer)
    
    return model

model = GoogLeNet(input_shape=(224,224,3), num_classes=1000)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_21 (Conv2D)             (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_8 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_21[0][0]']              
                                                                                            